In [4]:
import cv2
assert cv2.__version__[0] == '4', 'The fisheye module requires opencv version >= 3.0.0'
import numpy as np
import os
import glob
CHECKERBOARD = (7,7)
subpix_criteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)
calibration_flags = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC+cv2.fisheye.CALIB_FIX_SKEW
objp = np.zeros((1, CHECKERBOARD[0]*CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
_img_shape = None
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
images = glob.glob('*.png')
for fname in images:
    img = cv2.imread(fname)
    if _img_shape == None:
        _img_shape = img.shape[:2]
    else:
        assert _img_shape == img.shape[:2], "All images must share the same size."
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_FAST_CHECK+cv2.CALIB_CB_NORMALIZE_IMAGE)
    
    print(ret)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        cv2.cornerSubPix(gray,corners,(3,3),(-1,-1),subpix_criteria)
        imgpoints.append(corners)
N_OK = len(objpoints)
K = np.zeros((3, 3))
D = np.zeros((4, 1))
rvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
tvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
rms, _, _, _, _ = \
    cv2.fisheye.calibrate(
        objpoints,
        imgpoints,
        gray.shape[::-1],
        K,
        D,
        rvecs,
        tvecs,
        calibration_flags,
        (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-6)
    )
print("Found " + str(N_OK) + " valid images for calibration")
print("DIM=" + str(_img_shape[::-1]))
print("K=np.array(" + str(K.tolist()) + ")")
print("D=np.array(" + str(D.tolist()) + ")")
# DIM=(1088, 616)
# dim2 = (1088 * 2, 616 * 2)
# dim3 = (1088 * 3, 616 * 3)
# # K=np.array([[346.32115616796426, 0.0, 543.5], [0.0, 346.32115616796426, 307.5], [0.0, 0.0, 1.0]])
# # D=np.array([[0.0], [0.0], [0.0], [0.0]])
# balance = 0
# fname = images[0]
# img = cv2.imread(fname)
# dim1 = img.shape[:2][::-1]  #dim1 is the dimension of input image to un-distort
# assert dim1[0]/dim1[1] == DIM[0]/DIM[1], "Image to undistort needs to have same aspect ratio as the ones used in calibration"
# # if not dim2:
# # dim2 = dim1
# # if not dim3:
# # dim3 = dim1
# scaled_K = K * dim1[0] / DIM[0]  # The values of K is to scale with image dimension.
# scaled_K[2][2] = 1.0  # Except that K[2][2] is always 1.0
# # This is how scaled_K, dim2 and balance are used to determine the final K used to un-distort image. OpenCV document failed to make this clear!
# new_K = cv2.fisheye.estimateNewCameraMatrixForUndistortRectify(scaled_K, D, dim2, np.eye(3), balance=balance)
# map1, map2 = cv2.fisheye.initUndistortRectifyMap(scaled_K, D, np.eye(3), new_K, dim3, cv2.CV_16SC2)
# undistorted_img = cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
# cv2.imshow("undistorted", undistorted_img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

True
Found 1 valid images for calibration
DIM=(544, 308)
K=np.array([[181.20784053368962, 0.0, 269.26274741570063], [0.0, 180.34861809531762, 164.95661764906816], [0.0, 0.0, 1.0]])
D=np.array([[0.08869574884019396], [-0.06559255628891703], [0.07411420387674333], [-0.03169574352239552]])


In [6]:

# DIM=(1088, 616)
# K=np.array([[354.93201395880965, 0.0, 542.5266491557063], [0.0, 353.3228650112126, 325.66462617249334], [0.0, 0.0, 1.0]])
# D=np.array([[0.09107049072001831], [-0.08486396999427757], [0.1202734721798727], [-0.05674788013098452]])
DIM=(544, 308)
K=np.array([[181.20784053368962, 0.0, 269.26274741570063], [0.0, 180.34861809531762, 164.95661764906816], [0.0, 0.0, 1.0]])
D=np.array([[0.08869574884019396], [-0.06559255628891703], [0.07411420387674333], [-0.03169574352239552]])
def undistort(img_path, balance=0.0, dim2=None, dim3=None):
    img = cv2.imread(img_path)
    h,w = img.shape[:2]
    new_K = K.copy()
    # new_K[0][0] *= 0.5
    # new_K[1][1] *= 0.5
    map1, map2 = cv2.fisheye.initUndistortRectifyMap(K, D, np.eye(3), new_K, DIM, cv2.CV_16SC2)
    undistorted_img = cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)

    # a = 350
    # points = np.array([[[100, a], [200, a], [300, a], [400, a], [500, a], [600, a], [700, a], [800, a]]], np.float32)

    # undistortedPoints = cv2.undistortPoints(points, K, D);

    # for point in undistortedPoints:
    #     point = point[0]
    #     print(point)
    #     point[0] = point[0] * K[0][0] + K[0][2]
    #     point[1] = point[1] * K[1][1] + K[1][2]
    #     print(point)
        # for i in range(-3, 3):
        #     for j in range(-3, 3):
        #         undistorted_img[round(point[1]) + i][round(point[0]) + j] = [0, 255, 255]
    cv2.imwrite("undistorted_img.png", undistorted_img)
    cv2.imshow("undistorted", undistorted_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    # img = cv2.imread(img_path)
    # dim1 = img.shape[:2][::-1]  #dim1 is the dimension of input image to un-distort
    # print(dim1)
    # assert dim1[0]/dim1[1] == DIM[0]/DIM[1], "Image to undistort needs to have same aspect ratio as the ones used in calibration"
    # if not dim2:
    #     dim2 = dim1
    # if not dim3:
    #     dim3 = dim1
    # scaled_K = K * dim1[0] / DIM[0]  # The values of K is to scale with image dimension.
    # scaled_K[2][2] = 1.0  # Except that K[2][2] is always 1.0
    # # This is how scaled_K, dim2 and balance are used to determine the final K used to un-distort image. OpenCV document failed to make this clear!
    # new_K = cv2.fisheye.estimateNewCameraMatrixForUndistortRectify(scaled_K, D, dim2, np.eye(3), balance=0)
    # print(scaled_K)
    # print(new_K)
    # # new_K = scaled_K.copy()
    # # new_K[0][0] /= 4
    # # new_K[1][1] /= 4
    # map1, map2 = cv2.fisheye.initUndistortRectifyMap(scaled_K, D, np.eye(3), new_K, dim3, cv2.CV_16SC2)
    # undistorted_img = cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
    # cv2.imshow("undistorted", undistorted_img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
if __name__ == '__main__':
    # undistort("img14.png", 1)
    images = glob.glob('*.png')
    for fname in images:
        undistort(fname, 1)